In [40]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, Dataset, random_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import os

In [41]:
def get_matched_pairs(root_dir):
    """Tìm cặp file rung động và tốc độ, gán nhãn dựa trên tên."""
    pairs = []
    label_map = {'normal': 0, 'inner': 1, 'outer': 2, 'ball': 3}
    
    # Lấy danh sách tất cả file csv
    all_files = [f for f in os.listdir(root_dir) if f.endswith('.csv')]
    
    # Tìm các file rung động (vibration)
    vib_files = [f for f in all_files if 'vibration' in f.lower()]
    
    for v_file in vib_files:
        # Xác định nhãn sức khỏe
        assigned_label = None
        for key, val in label_map.items():
            if key in v_file.lower():
                assigned_label = val
                break
        
        if assigned_label is None: continue

        # Tìm file tốc độ tương ứng
        s_file = v_file.lower().replace('vibration', 'rpm')
        
        if s_file in [f.lower() for f in all_files]:
            v_path = os.path.join(root_dir, v_file)
            # Lấy tên file speed thực tế (giữ nguyên hoa thường)
            actual_s_file = [f for f in all_files if f.lower() == s_file][0]
            s_path = os.path.join(root_dir, actual_s_file)
            pairs.append((v_path, s_path, assigned_label))
            
    return pairs


def process_data_to_tensors(file_pairs, segment_length=2048):
    all_vib, all_speed, all_fault = [], [], []
    step = segment_length // 2 # Overlap 50%
    
    for v_path, s_path, label in file_pairs:
        print(f"Đang xử lý: {os.path.basename(v_path)} & {os.path.basename(s_path)}")
        
        # Đọc dữ liệu (7,680,000 mẫu)
        v_data = pd.read_csv(v_path).iloc[:, 0].values.astype(np.float32)
        s_data = pd.read_csv(s_path).iloc[:, 0].values.astype(np.float32)
        
        # Xác định độ dài tối thiểu của cả 2 file để tránh lệch index
        min_len = min(len(v_data), len(s_data))
        
        # Phân đoạn với Overlap 50%
        for start in range(0, min_len - segment_length + 1, step):
            end = start + segment_length
            
            # Trích xuất đoạn thô
            v_seg_raw = v_data[start:end]
            s_seg_raw = s_data[start:end] # Tốc độ giữ nguyên không chuẩn hóa
            
            # Kiểm tra độ dài chặt chẽ để tránh lỗi ValueError khi vstack
            if len(v_seg_raw) == segment_length and len(s_seg_raw) == segment_length:
                # 1. Chuẩn hóa Rung động theo từng mẫu [-1, 1] như bạn muốn
                v_min, v_max = np.min(v_seg_raw), np.max(v_seg_raw)
                v_seg_norm = 2 * (v_seg_raw - v_min) / (v_max - v_min + 1e-6) - 1
                
                # 2. Lưu dữ liệu (Tốc độ s_seg_raw được giữ nguyên giá trị RPM thực)
                all_vib.append(v_seg_norm.reshape(1, -1))
                all_speed.append(s_seg_raw.reshape(1, -1))
                all_fault.append(label)
            
    # Xếp chồng các mảng (Kích thước đồng nhất nên vstack sẽ không lỗi)
    vib_array = np.vstack(all_vib)
    speed_array = np.vstack(all_speed)
    
    # Chuyển đổi sang Tensor và thêm chiều channel (1) cho CNN 1D [cite: 319-322]
    return (torch.FloatTensor(vib_array).unsqueeze(1), 
            torch.FloatTensor(speed_array).unsqueeze(1), 
            torch.LongTensor(np.array(all_fault)))

class MDAMDataset(Dataset):
    def __init__(self, vib, speed, fault):
        self.vib, self.speed, self.fault = vib, speed, fault
    def __len__(self): return len(self.vib)
    def __getitem__(self, idx): return self.vib[idx], self.speed[idx], self.fault[idx]

In [42]:
class AttentionModule(nn.Module):
    def __init__(self, in_channels):
        super(AttentionModule, self).__init__()
        self.attention_path = nn.Sequential(
            nn.Conv1d(in_channels, 2, kernel_size=1),
            nn.ReLU(),
            nn.Conv1d(2, 1, kernel_size=1),
            nn.ReLU(),
            nn.Conv1d(1, 1, kernel_size=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        mask = self.attention_path(x)
        return x * mask + x

class EncoderBranch(nn.Module):
    def __init__(self):
        super(EncoderBranch, self).__init__()
        self.conv_blocks = nn.Sequential(
            # Block 1
            nn.Conv1d(1, 128, kernel_size=3, padding=1),
            nn.BatchNorm1d(128),
            nn.ELU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            
            # Block 2
            nn.Conv1d(128, 32, kernel_size=3, padding=1),
            nn.BatchNorm1d(32),
            nn.ELU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            
            # Block 3
            nn.Conv1d(32, 16, kernel_size=3, padding=1),
            nn.BatchNorm1d(16),
            nn.ELU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            
            # Block 4
            nn.Conv1d(16, 8, kernel_size=3, padding=1),
            nn.BatchNorm1d(8),
            nn.ELU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            
            # Block 5
            nn.Conv1d(8, 4, kernel_size=3, padding=1),
            nn.BatchNorm1d(4),
            nn.ELU(),
            nn.MaxPool1d(kernel_size=2, stride=2)
        )
        
        self.attention = AttentionModule(4)
        
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(4 * 64, 100),
            nn.ELU()
        )

    def forward(self, x):
        x = self.conv_blocks(x)
        x = self.attention(x)
        x = self.fc(x)
        return x

class MultiHeadEncoder(nn.Module):
    def __init__(self):
        super(MultiHeadEncoder, self).__init__()
        self.speed_encoder = EncoderBranch()
        self.fault_encoder = EncoderBranch()

    def forward(self, x):
        speed_features = self.speed_encoder(x)
        fault_features = self.fault_encoder(x)
        return speed_features, fault_features


In [43]:
class ClassificationHead(nn.Module):
    def __init__(self, input_dim=100, num_classes=3):
        super(ClassificationHead, self).__init__()
        
        self.classifier = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ELU(),
            
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        return self.classifier(x)
    

class SpeedDecoder(nn.Module):
    def __init__(self, feature_dim=100):
        super(SpeedDecoder, self).__init__()
        self.fc = nn.Sequential(nn.Linear(feature_dim, 256), nn.ELU())
        self.prepare = nn.Linear(256, 4 * 64) 
        
        self.deconv = nn.Sequential(
            nn.ConvTranspose1d(4, 32, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ELU(),
            nn.ConvTranspose1d(32, 16, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ELU(),
            nn.ConvTranspose1d(16, 8, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ELU(),
            nn.ConvTranspose1d(8, 4, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ELU(),
            nn.ConvTranspose1d(4, 1, kernel_size=3, stride=2, padding=1, output_padding=1)
        )

    def forward(self, r_speed):
        x = self.fc(r_speed)
        x = self.prepare(x).view(-1, 4, 64)
        return self.deconv(x)


In [44]:
class Decoder(nn.Module):
    """Module Decoder để tái tạo tín hiệu từ đặc trưng tốc độ và lỗi."""
    def __init__(self, feature_dim=100):
        super(Decoder, self).__init__()
        self.fc_init = nn.Sequential(
            nn.Linear(feature_dim * 2, 256),
            nn.ELU()
        )
        
        self.prepare_conv = nn.Linear(256, 4 * 64)
        
        self.deconv_blocks = nn.Sequential(
            nn.ConvTranspose1d(4, 128, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm1d(128),
            nn.ELU(),
            
            nn.ConvTranspose1d(128, 32, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm1d(32),
            nn.ELU(),
            
            nn.ConvTranspose1d(32, 16, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm1d(16),
            nn.ELU(),
            
            nn.ConvTranspose1d(16, 8, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm1d(8),
            nn.ELU(),
            
            nn.ConvTranspose1d(8, 1, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()
        )

    def forward(self, speed_features, fault_features):
        combined = torch.cat((speed_features, fault_features), dim=1)
        
        x = self.fc_init(combined)
        
        x = self.prepare_conv(x)
        x = x.view(x.size(0), 4, 64) 
        
        reconstructed_signal = self.deconv_blocks(x)
        return reconstructed_signal

In [45]:
class MDAMModel(nn.Module):
    def __init__(self, num_fault_classes=4):
        super(MDAMModel, self).__init__()
        
        # 1. Multi-head Encoder [cite: 319, 311]
        self.encoder = MultiHeadEncoder()
        
        # 2. Decoder cho Rung động (Vibration Reconstruction) [cite: 388]
        self.decoder_vib = Decoder(feature_dim=100)
        
        # 3. Speed Decoder (Speed Reconstruction - Theo yêu cầu của bạn)
        self.decoder_speed = SpeedDecoder(feature_dim=100)
        
        # 4. Fault Diagnosis Head (Phân loại 4 nhãn) [cite: 412]
        self.fd_head = ClassificationHead(input_dim=100, num_classes=num_fault_classes)

    def forward(self, x):
        # Trích xuất đặc trưng gỡ rối [cite: 115, 222]
        r_speed, r_fault = self.encoder(x)
        
        # Tái tạo tín hiệu rung động (x_hat) [cite: 231]
        vib_reconstructed = self.decoder_vib(r_speed, r_fault)
        
        # Tái tạo chuỗi tín hiệu tốc độ (z_hat)
        speed_reconstructed = self.decoder_speed(r_speed)
        
        # Chẩn đoán lỗi (y_hat) [cite: 230]
        fault_output = self.fd_head(r_fault)
        
        return vib_reconstructed, fault_output, speed_reconstructed

In [46]:
# Quy trình thực thi
TRAIN_ROOT = r"E:\5m\2nd\training_set"
TEST_ROOT = r"E:\5m\2nd\test_set"

# 1. Xử lý tập Train & Validation
train_pairs = get_matched_pairs(TRAIN_ROOT)
v_train, s_train, f_train = process_data_to_tensors(train_pairs)
full_ds = MDAMDataset(v_train, s_train, f_train)

train_size = int(0.8 * len(full_ds))
train_ds, val_ds = random_split(full_ds, [train_size, len(full_ds) - train_size])

# 2. Xử lý tập Test
test_pairs = get_matched_pairs(TEST_ROOT)
v_test, s_test, f_test = process_data_to_tensors(test_pairs)
test_ds = MDAMDataset(v_test, s_test, f_test)

# 3. Tạo Loaders (Batch size 128 theo bài báo [cite: 305])
train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=128, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=128, shuffle=False)

Đang xử lý: vibration_ball_0.csv & rpm_ball_0.csv
Đang xử lý: vibration_ball_1.csv & rpm_ball_1.csv
Đang xử lý: vibration_inner_0.csv & rpm_inner_0.csv
Đang xử lý: vibration_inner_1.csv & rpm_inner_1.csv
Đang xử lý: vibration_normal_0.csv & rpm_normal_0.csv
Đang xử lý: vibration_normal_1.csv & rpm_normal_1.csv
Đang xử lý: vibration_outer_0.csv & rpm_outer_0.csv
Đang xử lý: vibration_outer_1.csv & rpm_outer_1.csv
Đang xử lý: vibration_ball_2.csv & rpm_ball_2.csv
Đang xử lý: vibration_ball_3.csv & rpm_ball_3.csv
Đang xử lý: vibration_ball_4.csv & rpm_ball_4.csv
Đang xử lý: vibration_ball_5.csv & rpm_ball_5.csv
Đang xử lý: vibration_ball_6.csv & rpm_ball_6.csv
Đang xử lý: vibration_inner_2.csv & rpm_inner_2.csv
Đang xử lý: vibration_inner_3.csv & rpm_inner_3.csv
Đang xử lý: vibration_inner_4.csv & rpm_inner_4.csv
Đang xử lý: vibration_inner_5.csv & rpm_inner_5.csv
Đang xử lý: vibration_inner_6.csv & rpm_inner_6.csv
Đang xử lý: vibration_normal_2.csv & rpm_normal_2.csv
Đang xử lý: vibratio

In [47]:
def check_dataset_properties(dataset):
    print("--- KIỂM TRA THUỘC TÍNH DATASET ---")
    
    # 1. Tổng số mẫu
    total_samples = len(dataset)
    print(f"1. Tổng số mẫu (Segments): {total_samples}")
    
    # Lấy mẫu
    vib, speed, fault = dataset[0]
    
    # 2. Kiểm tra Shape
    # Rung động 
    print(f"2. Shape của Rung động (Vibration): {vib.shape}")
    # Tốc độ 
    print(f"   Shape của Tốc độ (Speed): {speed.shape}")
    print(f"   Nhãn lỗi (Fault Label): {fault.item()}")
    
    # 3. Kiểm tra Value Ranges
    print(f"3. Phạm vi giá trị Rung động: Min={vib.min():.4f}, Max={vib.max():.4f}")
    print(f"   Phạm vi giá trị Tốc độ: Min={speed.min():.4f}, Max={speed.max():.4f}")
    
    # 4. Kiểm tra Kiểu dữ liệu (Dtype)
    print(f"4. Dtype: Vib={vib.dtype}, Speed={speed.dtype}, Fault={fault.dtype}")
    
    # 5. Kiểm tra phân phối nhãn lỗi
    if hasattr(dataset, 'fault'):
        unique_labels, counts = torch.unique(dataset.fault, return_counts=True)
        label_names = {0: 'Normal', 1: 'Inner', 2: 'Outer', 3: 'Ball'}
        print("5. Phân phối nhãn lỗi:")
        for label, count in zip(unique_labels, counts):
            name = label_names.get(label.item(), "Unknown")
            print(f"   - {name} ({label.item()}): {count.item()} mẫu")

# Sử dụng:
check_dataset_properties(train_ds)

--- KIỂM TRA THUỘC TÍNH DATASET ---
1. Tổng số mẫu (Segments): 47987
2. Shape của Rung động (Vibration): torch.Size([1, 2048])
   Shape của Tốc độ (Speed): torch.Size([1, 2048])
   Nhãn lỗi (Fault Label): 3
3. Phạm vi giá trị Rung động: Min=-1.0000, Max=1.0000
   Phạm vi giá trị Tốc độ: Min=2191.0000, Max=2216.0652
4. Dtype: Vib=torch.float32, Speed=torch.float32, Fault=torch.int64


In [48]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"--- Đang sử dụng thiết bị: {device.type.upper()} " + (f"({torch.cuda.get_device_name(0)})" if device.type == 'cuda' else "") + " ---")

--- Đang sử dụng thiết bị: CUDA (NVIDIA GeForce GTX 1050 Ti) ---


In [49]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No CUDA")


2.9.1+cu126
True
NVIDIA GeForce GTX 1050 Ti


In [50]:
import torch.optim as optim

# 1. Cấu hình thiết bị và khởi tạo model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MDAMModel(num_fault_classes=4).to(device)

# 2. Định nghĩa hàm Loss và Optimizer [cite: 305]
mse_loss_fn = nn.MSELoss()           # Dùng cho cả tái tạo rung và tốc độ
ce_loss_fn = nn.CrossEntropyLoss()    # Dùng cho phân loại lỗi
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 3. Vòng lặp huấn luyện (Training Loop)
num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_faults = 0
    total_samples = 0
    
    for vib_clean, speed_true, fault_true in train_loader:
        # Chuyển dữ liệu lên GPU/CPU
        vib_clean = vib_clean.to(device)   # [Batch, 1, 2048]
        speed_true = speed_true.to(device) # [Batch, 1, 2048]
        fault_true = fault_true.to(device) # [Batch]
        
        # Forward pass
        vib_hat, fault_hat, speed_hat = model(vib_clean)
        
        # Tính toán các thành phần Loss [cite: 289, 436-438]
        loss_vib = mse_loss_fn(vib_hat, vib_clean)      # Loss tái tạo rung
        loss_speed = mse_loss_fn(speed_hat, speed_true) # Loss tái tạo tốc độ (MSE)
        loss_fault = ce_loss_fn(fault_hat, fault_true)   # Loss phân loại lỗi
        
        # Tổng hợp Loss (Multi-task Learning) [cite: 168]
        total_loss = loss_vib + loss_speed + loss_fault
        
        # Backward và cập nhật trọng số
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()
        
        # Thống kê
        running_loss += total_loss.item()
        _, predicted = torch.max(fault_hat, 1)
        correct_faults += (predicted == fault_true).sum().item()
        total_samples += fault_true.size(0)

    # Đánh giá nhanh trên tập Validation
    model.eval()
    val_acc = 0
    with torch.no_grad():
        for v_val, s_val, f_val in val_loader:
            v_val, f_val = v_val.to(device), f_val.to(device)
            _, f_hat, _ = model(v_val)
            val_acc += (torch.max(f_hat, 1)[1] == f_val).sum().item()
            
    print(f"Epoch [{epoch+1}/{num_epochs}] "
          f"Loss: {running_loss/len(train_loader):.4f} | "
          f"Fault Acc: {100*correct_faults/total_samples:.2f}% | "
          f"Val Acc: {100*val_acc/len(val_loader.dataset):.2f}%")

Epoch [1/50] Loss: 520140.6128 | Fault Acc: 61.08% | Val Acc: 61.34%
Epoch [2/50] Loss: 70016.6394 | Fault Acc: 66.76% | Val Acc: 65.21%
Epoch [3/50] Loss: 60659.3163 | Fault Acc: 68.99% | Val Acc: 67.40%
Epoch [4/50] Loss: 52246.1559 | Fault Acc: 71.57% | Val Acc: 68.18%
Epoch [5/50] Loss: 46656.4747 | Fault Acc: 73.68% | Val Acc: 70.09%
Epoch [6/50] Loss: 39265.5676 | Fault Acc: 75.91% | Val Acc: 73.02%
Epoch [7/50] Loss: 32906.0738 | Fault Acc: 77.94% | Val Acc: 75.76%
Epoch [8/50] Loss: 30115.4244 | Fault Acc: 79.58% | Val Acc: 76.86%
Epoch [9/50] Loss: 26609.6161 | Fault Acc: 81.22% | Val Acc: 73.12%
Epoch [10/50] Loss: 24933.5942 | Fault Acc: 81.80% | Val Acc: 75.88%
Epoch [11/50] Loss: 23903.4184 | Fault Acc: 83.50% | Val Acc: 78.24%
Epoch [12/50] Loss: 21986.9120 | Fault Acc: 84.48% | Val Acc: 79.95%
Epoch [13/50] Loss: 22229.0092 | Fault Acc: 85.48% | Val Acc: 78.47%
Epoch [14/50] Loss: 21499.1406 | Fault Acc: 86.15% | Val Acc: 74.84%
Epoch [15/50] Loss: 21856.2007 | Fault Acc

In [51]:
model.eval()
test_acc = 0
with torch.no_grad():
    for v_test, s_test, f_test in test_loader:
        v_test, f_test = v_test.to(device), f_test.to(device)
        _, f_hat, _ = model(v_test)
        test_acc += (torch.max(f_hat, 1)[1] == f_test).sum().item()
            
print(f"Test Acc: {100*test_acc/len(test_loader.dataset):.2f}%")

Test Acc: 70.34%
